In [ ]:

### Required Libraries ###
from datetime import datetime
from dateutil.relativedelta import relativedelta
from botocore.vendored import requests

### Functionality Helper Functions ###
def parse_float(n):
    """
    Securely converts a non-numeric value to float.
    """
    try:
        return float(n)
    except ValueError:
        return float("nan")
    except:
        return float("123456")

def get_cryptoprice(crypto, currency):
    """
    Retrieves the current price of BTC, BNB, ETH, ADA, XRP or SOL in top traded national currencies from the alternative.me Crypto API.
    """
    
    url = ""
    id = ""
    
    if crypto == "BTC":
        url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=" + currency
        id = "1"
    
    elif crypto == "BNB":
        url = "https://api.alternative.me/v2/ticker/BinanceCoin/?convert=" + currency
        id = "1839"
    
    elif crypto == "ETH":
        url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=" + currency
        id = "1027"
    
    elif crypto == "ADA":
        url = "https://api.alternative.me/v2/ticker/Cardano/?convert=" + currency
        id = "2010"
    
    elif crypto == "XRP":
        url = "https://api.alternative.me/v2/ticker/Ripple/?convert=" + currency
        id = "52"
    
    elif crypto == "SOL":
        url = "https://api.alternative.me/v2/ticker/Solana/?convert=" + currency
        id = "11733"

    response = requests.get(url)
    response_json = response.json()
    price_dollars = parse_float(response_json["data"][id]["quotes"][currency]["price"])
    return price_dollars


def build_validation_result(is_valid, violated_slot, message_content):
    """
    Defines an internal validation message structured as a python dictionary.
    """
    if message_content is None:
        return {"isValid": is_valid, "violatedSlot": violated_slot}

    return {
        "isValid": is_valid,
        "violatedSlot": violated_slot,
        "message": {"contentType": "PlainText", "content": message_content},
    }

def validate_data(birthday, dollars, currency, crypto, intent_request):
    """
    Validates the data provided by the user.
    """

    # Validate that the user is over 21 years old
    if birthday is not None:
        birth_date = datetime.strptime(birthday, "%Y-%m-%d")
        age = relativedelta(datetime.now(), birth_date).years
        if age < 21:
            return build_validation_result(
                False,
                "birthday",
                "You should be at least 21 years old to use this service, "
                "please provide a different date of birth.",
            )

    # Validate the investment amount, it should be > 0
    if dollars is not None:
        dollars = parse_float(
            dollars
        )  
    
    # Since parameters are strings it's important to cast values
        if dollars <= 0:
            return build_validation_result(
                False,
                "dollars",
                "The amount to convert should be greater than zero, "
                "please provide a correct amount in dollars to convert.",
            )
            
    # Validate if a correct curerency was passed
    if currency is not None:
        if currency not in ["USD", "JPY", "KRW", "EUR", "GBP", "CAD"]:
            return build_validation_result(
                False,
                "currency",
                "Sorry, I'm just able to convert from USD, JPY, KRW, EUR, GBP or CAD for now.",
            )
            
    # Validate if a correct cryptocurerency was passed
    if crypto is not None:
        if crypto not in ["BTC", "BNB", "ETH", "ADA", "XRP", "SOL"]:
            return build_validation_result(
                False,
                "crypto",
                "Sorry, I'm just able to convert from BTC, BNB, ETH, ADA, XRP or SOL for now.",
            )

    # A True results is returned if age or amount are valid
    return build_validation_result(True, None, None)


### Dialog Actions Helper Functions ###
def get_slots(intent_request):
    """
    Fetch all the slots and their values from the current intent.
    """
    return intent_request["currentIntent"]["slots"]

def elicit_slot(session_attributes, intent_name, slots, slot_to_elicit, message):
    """
    Defines an elicit slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "ElicitSlot",
            "intentName": intent_name,
            "slots": slots,
            "slotToElicit": slot_to_elicit,
            "message": message,
        },
    }

def delegate(session_attributes, slots):
    """
    Defines a delegate slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {"type": "Delegate", "slots": slots},
    }

def close(session_attributes, fulfillment_state, message):
    """
    Defines a close slot type response.
    """

    response = {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "Close",
            "fulfillmentState": fulfillment_state,
            "message": message,
        },
    }

    return response


### Intents Handlers ###
def convert_dollars(intent_request):
    """
    Performs dialog management and fulfillment for converting from dollars to cryptocurerency.
    """

    # Gets slots' values
    birthday = get_slots(intent_request)["birthday"]
    dollars = get_slots(intent_request)["dollars"]
    currency = get_slots(intent_request)["currency"]
    crypto = get_slots(intent_request)["crypto"]

    # Explicitly parse currency as string and make it uppercase
    currency = str(currency).upper()
    
    # Explicitly parse currency as string and make it uppercase
    crypto = str(crypto).upper()

    # Gets the invocation source, for Lex dialogs "DialogCodeHook" is expected.
    source = intent_request["invocationSource"]

    if source == "DialogCodeHook":
        # This code performs basic validation on the supplied input slots.

        # Gets all the slots
        slots = get_slots(intent_request)

        # Validates user's input using the validate_data function
        validation_result = validate_data(birthday, dollars, currency, crypto, intent_request)

        # If the data provided by the user is not valid,
        # the elicitSlot dialog action is used to re-prompt for the first violation detected.
        if not validation_result["isValid"]:
            slots[validation_result["violatedSlot"]] = None  # Cleans invalid slot

            # Returns an elicitSlot dialog to request new data for the invalid slot
            return elicit_slot(
                intent_request["sessionAttributes"],
                intent_request["currentIntent"]["name"],
                slots,
                validation_result["violatedSlot"],
                validation_result["message"],
            )

        # Fetch current session attributes
        output_session_attributes = intent_request["sessionAttributes"]

        # Once all slots are valid, a delegate dialog is returned to Lex to choose the next course of action.
        return delegate(output_session_attributes, get_slots(intent_request))

    # Get the current price of BTC, BNB, ETH, ADA, XRP or SOL and make the conversion from top traded national currencies.
    
    crypto_value = parse_float(dollars) / get_cryptoprice(crypto, currency)
    crypto_value = round(crypto_value, 4)

    # Return a message with conversion's result.
    return close(
        intent_request["sessionAttributes"],
        "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """ Thank you for your information;
            you can get {} {} for your ${} {}.
            """.format(
                crypto_value, crypto, dollars, currency
            ),
        },
    )


### Intents Dispatcher ###
def dispatch(intent_request):
    """
    Called when the user specifies an intent for this bot.
    """

    # Get the name of the current intent
    intent_name = intent_request["currentIntent"]["name"]

    # Dispatch to bot's intent handlers
    if intent_name == "convertDollarsInt":
        return convert_dollars(intent_request)

    raise Exception("Intent with name " + intent_name + " not supported")


### Main Handler ###
def lambda_handler(event, context):
    """
    Route the incoming request based on intent.
    The JSON body of the request is provided in the event slot.
    """

    return dispatch(event)